In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install python_speech_features


  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5869 sha256=e37e0a1074537158016c6eab48935f1edba50986c922682aa9f035ac5b942f02
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [ ]:
!pip install noisereduce


In [ ]:

import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
os.listdir(path='/content/gdrive/MyDrive/NLP/Emotions')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = '/content/gdrive/MyDrive/NLP/Emotions'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)


7442

In [ ]:
!pip install SpeechRecognition


In [ ]:
#Now Cleaning Step is Performed where:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [ ]:
import soundfile
import librosa
import numpy as np

def extract_feature(file_name):#, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        features = []
        # Compute STFT
        stft = np.abs(librosa.stft(X))
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            features.append(mfccs)

    return np.concatenate(features)


In [ ]:
#Emotions in the RAVDESS dataset to be classified Audio Files based on .
emotions={
  '01':'SAD',
  '02':'ANG',
  '03':'DIS',
  '04':'NEU',
  '05':'HAP',
  '06':'FEA'
}
#These are the emotions User wants to observe more :
observed_emotions=['SAD', 'ANG', 'DIS', 'NEU','HAP','FEA']

In [ ]:
#Split the dataset
#Load the data and extract features for each sound file
from glob import glob
import os
import glob
x,y=[],[]
def load_data(test_size=0.33):

    answer = 0
    for file in glob.glob('/content/gdrive/MyDrive/NLP/clean/*.wav'):
        file_name=os.path.basename(file)
        emotion=file_name.split("_")[2]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [ ]:
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.33)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


(4986, 40) (2456, 40) (4986, 2) (2456, 2)
(4986,) (2456,)
1083_IWW_HAP_XX.wav
1063_DFA_FEA_XX.wav
1023_IEO_FEA_LO.wav
1073_IEO_ANG_HI.wav
1045_IOM_SAD_XX.wav
1003_WSI_HAP_XX.wav
1049_IEO_SAD_HI.wav
1040_WSI_NEU_XX.wav
1041_TSI_HAP_XX.wav
1030_TIE_HAP_XX.wav
1005_ITS_FEA_XX.wav
1058_WSI_FEA_XX.wav
1004_IEO_ANG_HI.wav
1031_IOM_NEU_XX.wav
1048_ITH_FEA_XX.wav
1021_DFA_SAD_XX.wav
1060_ITH_SAD_XX.wav
1007_TSI_ANG_XX.wav
1006_IEO_SAD_LO.wav
1091_ITS_DIS_XX.wav
1059_DFA_SAD_XX.wav
1050_IOM_NEU_XX.wav
1055_IEO_ANG_HI.wav
1009_IWW_FEA_XX.wav
1052_IWW_HAP_XX.wav
1014_TIE_NEU_XX.wav
1080_IWL_FEA_XX.wav
1013_TAI_SAD_XX.wav
1082_IWL_NEU_XX.wav
1044_TIE_HAP_XX.wav
1063_WSI_DIS_XX.wav
1038_TIE_FEA_XX.wav
1087_TIE_DIS_XX.wav
1013_TSI_ANG_XX.wav
1014_WSI_FEA_XX.wav
1006_IWL_FEA_XX.wav
1020_MTI_NEU_XX.wav
1083_TSI_SAD_XX.wav
1030_IWW_HAP_XX.wav
1002_ITH_NEU_XX.wav
1010_TIE_FEA_XX.wav
1052_IEO_ANG_HI.wav
1054_DFA_SAD_XX.wav
1009_TIE_FEA_XX.wav
1073_ITS_DIS_XX.wav
1071_ITH_ANG_XX.wav
1054_IEO_SAD_HI.wav
10

In [ ]:
#Get the shape of the training and testing datasets
# print((x_train.shape[0], x_test.shape[0]))
print((x_train[0], x_test[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(array([-3.6612375e+02,  1.1611234e+02,  3.6821342e+01,  2.4848091e+01,
        8.8491335e+00,  1.4262024e+00, -5.6962214e+00, -1.0569816e+01,
       -4.8323565e+00,  2.1905196e-01,  1.0040426e+00,  1.7446524e+00,
       -4.5957422e+00,  4.4904032e-01, -4.3665781e+00, -7.6925263e+00,
       -1.0500817e+00, -1.8648098e+00, -3.1442821e+00, -2.8669569e+00,
       -3.8024020e+00, -1.5841887e+00, -1.2783916e-01,  1.8475616e-01,
       -1.0088890e+00, -4.3700257e-01, -1.0497243e+00,  9.8161757e-01,
        1.0312353e+00,  1.1238776e+00,  2.5561142e-03, -7.6619908e-02,
        4.8503823e+00,  3.0555139e+00,  7.6161689e-01, -8.5774076e-01,
       -3.6561435e-01,  1.5788814e-02, -1.8207607e-01, -3.2750211e+00],
      dtype=float32), array([-318.04877   ,   97.72982   ,   24.048115  ,   12.561897  ,
         10.749334  ,    8.370912  ,  -14.030314  ,  -11.880463  ,
         -3.1392086 ,   -6.6673775 ,   -4.0251045 ,   -0.5695503 ,
         -9.344196  ,   -5.241774  ,    4.7175846 ,   -7.920106  

In [ ]:

# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 42.26%


In [ ]:

#SAVING THE MODEL
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = "Emotion_Voice_Detection_Model.pkl"

with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#predicting :
y_pred=Emotion_Voice_Detection_Model.predict(x_test)
y_pred

array(['ANG', 'HAP', 'HAP', ..., 'HAP', 'SAD', 'NEU'], dtype='<U3')

In [ ]:
print(accuracy_score(y_pred,y_test))

0.4226384364820847


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'alpha': [0.0001, 0.001, 0.01],
    'epsilon': [1e-08, 1e-06, 1e-05],
    'hidden_layer_sizes': [(100,), (200,), (300,)],
    'learning_rate': ['constant', 'adaptive']
}

# Initialize the MLPClassifier
mlp = MLPClassifier(max_iter=500)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Best Parameters: {'alpha': 0.001, 'epsilon': 1e-08, 'hidden_layer_sizes': (300,), 'learning_rate': 'constant'}
Accuracy: 0.44258957654723124


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Data Preprocessing
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

if len(encoded_labels) == 0:
    print("Error: No labels found after encoding.")
else:
    encoded_labels = to_categorical(encoded_labels)

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

    # Model Building
    model = Sequential([
        Dense(128, activation='relu', input_shape=(features.shape[1],)),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Hyperparameter Tuning
    # Train the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    # Evaluation
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)


Error: No labels found after encoding.
